In [1]:
import random
random.seed(10)

import os
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
#from keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping

!pip install wandb
import wandb
from wandb.keras import WandbCallback
!pip install wandb --upgrade

In [2]:
# Prepare the dataset for training and testing
def prepare_dataset(DATA_DIR="inaturalist_12K", augment_data=False, batch_size=100):
    train_dir = os.path.join(DATA_DIR, "train")
    test_dir = os.path.join(DATA_DIR, "val")

    if augment_data:
        train_datagen = ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
        test_datagen = ImageDataGenerator(rescale=1./255)

    else:
        train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="training")
    val_generator = train_datagen.flow_from_directory(train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='categorical', subset="validation")
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(image_size, image_size), batch_size=batch_size)
    
    return train_generator, val_generator, test_generator

In [3]:
input_shape = (224,224,3)
image_size = input_shape[0]
# Comment the next line if you want to sweep over different models
# base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')

def train():
    
    config_defaults = {
        'freeze_before' : 50,
        'models' : 'IV3',
        'dropout': 0.2,
        'batch_size': 64,
        'dense_size': 100,
        'num_classes': 10,
        'epochs' : 5,
        'augment_data': True,
        'lr': 0.001
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    
    # Defining models and run-name
    if config.models == 'IV3':
        base_model =  keras.applications.InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncV3__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'IRNV2':
        base_model =  keras.applications.InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_IncResNetV2__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'RN50':
        base_model =  keras.applications.ResNet50(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_ResNet50__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)
    elif config.models == 'XCP':
        base_model =  keras.applications.Xception(input_shape = input_shape, include_top = False, weights = 'imagenet')
        wandb.run.name = 'model_Xception__num_dense_'+ str(config.dense_size)+'__bs_'+str(config.batch_size)

        
    base_model.trainable = False

    stack_model = Sequential()
    stack_model.add(base_model)
    stack_model.add(Flatten())
    stack_model.add(Dense(config.dense_size, activation='relu'))
    stack_model.add(Dropout(config.dropout))
    stack_model.add(Dense(config.num_classes, activation='softmax'))
    
    final_model = stack_model
    final_model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")
    
    train_generator, val_generator, test_generator = prepare_dataset(augment_data=config.augment_data, batch_size=config.batch_size)
    
    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs=config.epochs,
                    validation_data=val_generator,
                    callbacks=[WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - config.freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False
    
    final_model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="accuracy")

    final_model.fit(train_generator,
                    batch_size = config.batch_size,
                    epochs = 7,
                    validation_data=val_generator,
                    callbacks = [WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

In [7]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        
        'freeze_before' : {
            'values': [50, 70, 100]
        },
        'models' :{
            'values' : ['IV3','IRNV2', 'RN50', 'XCP']
        },
        'dropout': {
            'values': [0, 0.1, 0.2, 0.3]
        },     
        'batch_size': {
            'values': [32, 64]
        },
        'dense_size':{
            'values': [64, 100, 128]
        },
        'augment_data': {
            "values": [False, True]
        },
        'epochs': {
            "values": [5]
        },
        'lr': {
            "values": [0.005, 0.001, 0.0001]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="DL_CS6910_Assignment2")

Create sweep with ID: 4pxgyjqv
Sweep URL: https://wandb.ai/cs21m029_keyur_raval/DL_CS6910_Assignment2/sweeps/4pxgyjqv


In [8]:
wandb.agent(sweep_id, train, count = 10)

wandb: Agent Starting Run: cltbbdao with config:
wandb: 	augment_data: True
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	freeze_before: 70
wandb: 	lr: 0.0001
wandb: 	models: IRNV2


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
141/141 [==============================] - 148s 962ms/step - loss: 1.2509 - accuracy: 0.6058 - val_loss: 1.0079 - val_accuracy: 0.6727
Epoch 2/5
141/141 [==============================] - 131s 927ms/step - loss: 1.0127 - accuracy: 0.6694 - val_loss: 1.0108 - val_accuracy: 0.6577
Epoch 3/5
141/141 [==============================] - 132s 938ms/step - loss: 0.9949 - accuracy: 0.6793 - val_loss: 0.9458 - val_accuracy: 0.7087
Epoch 4/5
141/141 [==============================] - 133s 943ms/step - loss: 0.9384 - accuracy: 0.6936 - val_loss: 1.0335 - val_accuracy: 0.6697
Epoch 5/5
141/141 [==============================] - 133s 941ms/step - loss: 0.8975 - accuracy: 0.7053 - val_loss: 0.8972 - val_accuracy: 0.7087
Epoch 1/7
141/141 [==============================] - 147s 970ms/step - loss: 0.9077 - accuracy: 0.7069 - val_loss: 0.8444 - val_accuracy: 0.7287
Ep

accuracy,▁▃▃▄▄▄▅▆▆▇▇█
epoch,▁▂▃▅▆▁▂▃▅▆▇█
loss,█▆▆▅▅▅▄▃▃▂▁▁
val_accuracy,▂▁▄▂▄▅▆▅▇▇▇█
val_loss,▇▇▆█▄▃▂▃▂▁▂▁
accuracy,0.84289
best_epoch,6
best_val_loss,0.76197
epoch,6
loss,0.46868
val_accuracy,0.77277


wandb: Agent Starting Run: zbkeq4rf with config:
wandb: 	augment_data: True
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	freeze_before: 100
wandb: 	lr: 0.0001
wandb: 	models: IV3


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
141/141 [==============================] - 138s 940ms/step - loss: 1.3597 - accuracy: 0.5680 - val_loss: 1.1464 - val_accuracy: 0.6236
Epoch 2/5
141/141 [==============================] - 130s 922ms/step - loss: 1.0388 - accuracy: 0.6579 - val_loss: 1.0387 - val_accuracy: 0.6547
Epoch 3/5
141/141 [==============================] - 127s 903ms/step - loss: 0.9810 - accuracy: 0.6779 - val_loss: 1.0640 - val_accuracy: 0.6476
Epoch 4/5
141/141 [==============================] - 126s 895ms/step - loss: 0.9434 - accuracy: 0.6852 - val_loss: 1.0971 - val_accuracy: 0.6547
Epoch 5/5
141/141 [==============================] - 126s 896ms/step - loss: 0.8755 - accuracy: 0.7064 - val_loss: 1.0237 - val_accuracy: 0.6617
Epoch 1/7
141/141 [==============================] - 135s 913ms/step - loss: 0.9706 - accuracy: 0.6778 - val_loss: 0.9433 - val_accuracy: 0.6887
Ep

...

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
282/282 [==============================] - 136s 469ms/step - loss: 4.7640 - accuracy: 0.1503 - val_loss: 2.0717 - val_accuracy: 0.2142
Epoch 2/5
282/282 [==============================] - 131s 465ms/step - loss: 2.2512 - accuracy: 0.1388 - val_loss: 2.2310 - val_accuracy: 0.1331
Epoch 3/5
282/282 [==============================] - 131s 463ms/step - loss: 2.2358 - accuracy: 0.1287 - val_loss: 2.2092 - val_accuracy: 0.1411
Epoch 4/5
282/282 [==============================] - 130s 462ms/step - loss: 2.2496 - accuracy: 0.1361 - val_loss: 2.2200 - val_accuracy: 0.1421
Epoch 5/5
282/282 [==============================] - 130s 461ms/step - loss: 2.2620 - accuracy: 0.1222 - val_loss: 2.2396 - val_accuracy: 0.1271
Epoch 1/7
282/282 [==============================] - 138s 472ms/step - loss: 2.3385 - accuracy: 0.0970 - val_loss: 2.3028 - val_accuracy: 0.1001
Ep

accuracy,█▇▅▆▅▂▁▁▁▂▁▁
epoch,▁▂▃▅▆▁▂▃▅▆▇█
loss,█▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▃▄▄▃▁▁▁▁▁▁▁
val_loss,▁▆▅▅▆███████
accuracy,0.09256
best_epoch,0
best_val_loss,2.0717
epoch,6
loss,2.30361
val_accuracy,0.1001


wandb: Agent Starting Run: do6xrfrw with config:
wandb: 	augment_data: False
wandb: 	batch_size: 32
wandb: 	dense_size: 100
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	freeze_before: 70
wandb: 	lr: 0.001
wandb: 	models: RN50


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/5
282/282 [==============================] - 103s 345ms/step - loss: 2.4815 - accuracy: 0.0937 - val_loss: 2.3026 - val_accuracy: 0.1001
Epoch 2/5
282/282 [==============================] - 97s 345ms/step - loss: 2.3028 - accuracy: 0.0971 - val_loss: 2.3026 - val_accuracy: 0.1001
Epoch 3/5
282/282 [==============================] - 98s 346ms/step - loss: 2.3028 - accuracy: 0.0953 - val_loss: 2.3026 - val_accuracy: 0.1001
Epoch 4/5
282/282 [==============================] - 91s 322ms/step - loss: 2.3029 - accuracy: 0.0943 - val_loss: 2.3026 - val_accuracy: 0.1001
Epoch 5/5
282/282 [==============================] - 94s 333ms/step - loss: 2.3028 - accuracy: 0.0962 - val_loss: 2.3026 - val_accuracy: 0.1011
Epoch 1/7
282/282 [==============================] - 109s 363ms/step - loss: 2.3694 - accuracy: 0.0950 - val_loss: 2.3026 - val_accuracy: 0.1001
Epoch 

accuracy,▃█▅▄▇▅▂▅▄▁▅▆
epoch,▁▂▃▅▆▁▂▃▅▆▇█
loss,█▁▁▁▁▄▁▁▁▁▁▁
val_accuracy,▁▁▁▁█▁▁▁▁▁▁▁
val_loss,▆▅▁▄▅▇▂▇▆█▄▄
accuracy,0.09611
best_epoch,2
best_val_loss,2.30258
epoch,6
loss,2.30281
val_accuracy,0.1001
